# SCRAPING

In [190]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
from tqdm.notebook import tqdm_notebook
import pandas as pd
import os
import time
import warnings
warnings.filterwarnings("ignore")

In [20]:
def readpage(url):
    req = requests.get(url, verify=False)
    page = bs(req.content, 'lxml')
    return page

## TurnBackHoax.id

In [55]:
driver = webdriver.Edge('msedgedriver.exe')

In [56]:
website = 'https://turnbackhoax.id/'
driver.get(website)

In [57]:
hoax.find('a', {'rel':'bookmark'})['href']

'https://turnbackhoax.id/2021/07/09/salah-campuran-air-panas-dan-lemon-rahasia-vietnam-tak-alami-kasus-kematian-akibat-covid-19/'

In [58]:
%%time
judul_hoax = []
date_hoax = []
link_hoax = []
for i in tqdm.notebook(range(10)):
    hoax = bs(driver.page_source, 'lxml')
    # Judul
    header = hoax.find_all('a', {'rel':'bookmark'})
    judul = [j.get_text().strip() for j in header]
    judul_hoax.extend(judul)
    
    # Link
    link = [l['href'] for l in header]
    link_hoax.extend(link)
    
    # Date
    date = hoax.find_all('span', {'class':'mh-meta-date updated'})
    date = [d.get_text() for d in date]
    date_hoax.extend(date)
    
    next_page = driver.find_element_by_class_name('next')
    next_page.click()

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

Wall time: 11.3 s


In [59]:
hoax_df = pd.DataFrame(list(zip(judul_hoax, date_hoax, link_hoax)))

## CNN Indonesia

In [222]:
driver = webdriver.Edge('msedgedriver.exe')

In [223]:
%%time
website = 'https://www.cnnindonesia.com/search?query=covid'
driver.get(website)

Wall time: 3.29 s


In [224]:
%%time
judul_cnn = []
link_cnn = []
date_cnn = []

# Page
for i in tqdm_notebook(range(200)):
    next_page = driver.find_element_by_class_name('gtm_button_indeks')
    next_page.click()
    time.sleep(0.5)

# Define Container
cnn = bs(driver.page_source, 'lxml')
container = cnn.find('div', {'class':'list media_rows middle'})
# Get Articles In Container
articles = container.findChildren('article')

# Get Judul
for article in articles:
    judul_cnn.append(article.find('h2',{'class':'title'}).get_text())

# Get Links
for article in articles:
    childlink = article.findChildren('a')
    link = childlink[0]['href']
    link_cnn.append(link)

# Dates
for link in link_cnn:
    page = readpage(link)
    date = page.find('div', {'class':'date'}).get_text().strip()
    date_cnn.append(date)
    
print('Complete!')


Complete!
Wall time: 18min 45s


In [231]:
# Make into DF
cnn_df = pd.DataFrame(list(zip(judul_cnn, date_cnn, link_cnn)))

In [233]:
# Put into CSV
cnn_df.to_csv('CNN_data.csv')

## Detik

In [ ]:
driver = webdriver.Edge('msedgedriver.exe')

In [279]:
website = 'https://www.detik.com/search/searchall?query=covid&siteid=2&sortby=time&page=1'
driver.get(website)

In [ ]:
%%time
judul_detik = []
date_detik = []
link_detik = []

# Page Open
for i in tqdm_notebook(range(150)):
    page = bs(driver.page_source, 'lxml')
    
    # Define Group
    group = page.find('div', {'class':'list media_rows list-berita'})
    articles = group.findChildren('article', recursive=False)
        
    # Judul
    judul = [article.find('h2', {'class':'title'}).get_text().strip() for article in articles]
    judul_detik.extend(judul)
    
    # Date
    for article in articles:
        date = article.find('span', {'class':'date'}).text
        date_detik.append(date)
    
    # Links
    links = [article.find('a')['href'] for article in articles]
    link_detik.extend(links)
    
    
    try:
        next_page = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/a[7]')
        next_page.click()
    except:
        next_page = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/a[6]')
        next_page.click()
        

In [275]:
# Make into DF
dtk_df = pd.DataFrame(list(zip(judul_detik, date_detik, link_detik)))
dtk_df.columns = ['judul', 'tanggal', 'link']

In [ ]:
# Put into CSV
dtk_df.to_csv('detik_data.csv', columns=)